# Stochastic Blockmodeling of Layered Graphs
**Data**: Time-stamped movements of about 200 users in eastern Germany as measured through GPS, collapsed into 50 or 100 network nodes.

**Analysis**: Detection of mobility patternsusing stochastic blockmodeling using <font face='Courier'>graph-tool</font>.
## Dependencies

In [2]:
import numpy as np
import pandas as pd
import pickle
from graph_tool.all import *

## Data Preparation
Load edgelist for network with either 50 or 100 nodes. Each directed link corresponds to a user's movement from one point in the region (network node) to another:

In [3]:
edgelist = pd.read_csv('../data/edgelist_chemnitz_dec4_distTrue_cluAgglomerative50.txt', sep='\t')
#edgelist = pd.read_csv('../data/edgelist_chemnitz_dec4_distTrue_cluAgglomerative100.txt', sep='\t')

Get the hour of the week for the beginning of each movement. Monday 0am is hour 0, Sunday 11pm is hour 167:

In [4]:
edgelist['time_begin'] = pd.to_datetime(edgelist['time_begin'])
edgelist['time_end'] = pd.to_datetime(edgelist['time_end'])

In [5]:
edgelist['hourofweek'] = edgelist['time_begin'].dt.dayofweek*24+edgelist['time_begin'].dt.hour

In [6]:
edgelist.head()

,time_begin,time_end,cluster_id_begin,cluster_id_end,type,speed,hourofweek
0,2015-11-04 10:12:28,2015-11-04 11:06:35,0,0,foot,0.64,58
1,2015-11-04 11:06:35,2015-11-04 11:06:43,0,0,car,785.12,59
2,2015-11-04 12:56:46,2015-11-04 12:57:59,0,0,foot,2.59,60
3,2015-11-04 12:57:59,2015-11-04 13:11:34,0,0,foot,1.95,60
4,2015-11-04 13:11:34,2015-11-04 13:14:24,0,0,car,22.64,61


Sample edgelist to speed up computation:

In [7]:
sample = 100
edgelist = edgelist.ix[np.random.choice(edgelist.index.values, sample)]

/srv/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


Subset edgelist to work with:

In [8]:
edgelist_hourofweek = edgelist[['cluster_id_begin', 'cluster_id_end', 'hourofweek']]

In [9]:
edgelist_hourofweek.head()

,cluster_id_begin,cluster_id_end,hourofweek
18288,11,11,109
10109,0,6,105
19436,17,16,32
23635,35,35,113
10293,0,6,58


Above table contains multiple entries (same edge at same time). Get weighted edge list:

In [10]:
edgelist_hourofweek_weight = edgelist_hourofweek.groupby(['cluster_id_begin', 'cluster_id_end', 'hourofweek']).size().reset_index()
edgelist_hourofweek_weight.rename(columns={0:'weight'}, inplace=True)

In [11]:
edgelist_hourofweek_weight.head()

,cluster_id_begin,cluster_id_end,hourofweek,weight
0,0,0,15,2
1,0,0,33,1
2,0,0,37,2
3,0,0,41,1
4,0,0,53,1


Construct multigraph with <font face='Courier'>hourofweek</font> as edge property:

In [12]:
g = Graph(directed=True)
hourofweek = g.new_edge_property('int')
g.add_edge_list(edgelist_hourofweek.values, eprops=[hourofweek])
g.edge_properties['hourofweek'] = hourofweek

In [13]:
print(g)

<Graph object, directed, with 43 vertices and 100 edges at 0x7f640aac2400>


In [14]:
g.list_properties()

hourofweek     (edge)    (type: int32_t)


Construct multigraph with <font face='Courier'>hourofweek</font> and <font face='Courier'>weight</font> as edge properties:

In [15]:
g_weight = Graph(directed=True)
hourofweek = g_weight.new_edge_property('int')
weight = g_weight.new_edge_property('int')
g_weight.add_edge_list(edgelist_hourofweek_weight.values, eprops=[hourofweek, weight])
g_weight.edge_properties['hourofweek'] = hourofweek
g_weight.edge_properties['weight'] = weight

*Q1: Is the weighted graph constructed correctly?*

**A1: Looks fine!**

In [16]:
print(g_weight)

<Graph object, directed, with 43 vertices and 91 edges at 0x7f640aac2fd0>


In [17]:
g_weight.list_properties()

hourofweek     (edge)    (type: int32_t)
weight         (edge)    (type: int32_t)


## Data Analysis
The goal is to perform an analysis as in fig.8 of "<a href='https://arxiv.org/abs/1504.02381'>Inferring the mesoscale structure of layered, edge-valued, and time-varying networks"</a> (Peixoto, 2015). The idea is to use the hour of the week as the time of weighted edges and detect the change points in aggregate mobility.
### Are layers informative?
To test this, we (a) infer the nested blockmodel for a weighted graph with collapsed edge time, (b) infer the nested blockmodel for a weighted graph with edge time as layers, and (c) compare the posterior odds ratio. Degrees are assumed to be correlated. (***COMMENT: 'deg_corr' means degree correction, not degree correlation.***)
#### (a) Nested blockmodel for weighted graph with collapsed edge time

In [32]:
state_nested_covariates = minimize_nested_blockmodel_dl(g_weight, layers=True, state_args=dict(ec=g_weight.ep.hourofweek, layers=False, recs=[g_weight.ep.weight], rec_types=['discrete-poisson']), deg_corr=True)
#pickle.dump(state_nested_covariates, open('state_nested_covariates.pickle', 'wb'))

In [33]:
#state_nested_covariates = pickle.load(open('state_nested_covariates.pickle', 'rb'))

*Q1: To use both edge properties (hourofday and weight), can the module be called like that?*

**A: Yes it can. This amounts to a layered weighted graph.**

*Q2: Is the covariate type correct ('discrete-poisson')?*

**A: It depends. For discrete covariates, there are three possible models: "discrete-geometric", "discrete-binomial" and "discrete-poisson". Which one to use is a model selection question, and depends on data. Generically you should use the one that yields the largest posterior (smallest description length), but usually it is clear which one is best by inspecting the nature of the weights.**

**But there is a sublety here: If the weights are just edge multiplicities of a multigraph, you should probably not pass them as covariates, since the underlying SBM already models multigraphs. Instead, you should use the 'eweight' parameter, as such:**

In [34]:
state_nested_covariates = minimize_nested_blockmodel_dl(g_weight, layers=True, state_args=dict(ec=g_weight.ep.hourofweek, layers=False, eweight=g_weight.ep.weight), deg_corr=True)

***However, it is important to make users aware that the 'eweight' parameter is only meant for edge multiplicities. Passing to it arbitrary "weights", i.e. real values or negative values, is incorrect. For this kind of arbitrary covariate, ther 'recs' parameter should be used instead.***


*Q3: I'm confused about the two layers parameters. Is it that the first one specifies that observed data has edge weights and the second (state_arg) specifies if that information is collapsed?
A: "Note the different meanings of the two 'layers' parameters below: The first enables the use of LayeredBlockState, and the second selects the 'edge layers' version (instead of 'edge covariates')." (https://graph-tool.skewed.de/static/doc/demos/inference/inference.html#layered-networks)*

**This is indeed prone to confusion. The first 'layers=True' simply activates the use of layered SBM, whereas the second one selects the ***kind*** of layered SBM: 'layers=False' means that the collapsed network is generated first, and then the edges are distributed, whereas 'layers=True' means that the layers are generated independently.***

#### (b) Nested blockmodel for weighted graph with edge time as layers


In [35]:
state_nested_layers = minimize_nested_blockmodel_dl(g_weight, layers=True, state_args=dict(ec=g_weight.ep.hourofweek, layers=True, eweight=g_weight.ep.weight), deg_corr=True)
#pickle.dump(state_nested_layers, open('state_nested_layers.pickle', 'wb'))

In [36]:
#state_nested_layers = pickle.load(open('state_nested_layers.pickle', 'rb'))

#### (c) Model selection

In [37]:
state_nested_covariates.entropy()-state_nested_layers.entropy()

-1561.7338865161232

*Q4: So layers are not informative because the difference is negative?*

***The 'entropy()' method returs the description length of the fit, i.e. the negative log-probability. Thus, the model with the smallest value should be selected. A negative difference, like in the above, means that the first model should be selected (state_nested_covariates).***

***Note, however, that both models explain the layers, they only do so differently. So this comparison cannot be used to tell if the layers are informative or not. Instead, one would need to compare with a model that discards this information altogether, and just distributes the layers randomly:***

In [38]:
state_nested = minimize_nested_blockmodel_dl(g_weight, state_args=dict(eweight=g_weight.ep.weight), deg_corr=True)
#pickle.dump(state_nested_layers, open('state_nested_layers.pickle', 'wb'))

***We cannot directly compare state_nested with state_nested_covariates, since the former does not explain the layers. We must add to it the probability that the layer configuration is obtained randomly (Eq. A2 in the 2015 paper):***

In [52]:
from scipy.special import gammaln
from numpy import bincount

ret = condensation_graph(g_weight, g_weight.vertex_index, eweight=g_weight.ep.weight)
ecount = ret[3]

S_null = state_nested.entropy() + gammaln(g_weight.ep.weight.fa.sum() + 1) - gammaln(bincount(g_weight.ep.hourofweek.fa, weights=g_weight.ep.weight.fa) + 1).sum() + gammaln(g_weight.ep.weight.fa + 1).sum() - gammaln(ecount.fa + 1).sum()

print(S_null-state_nested_covariates.entropy())

83.5376006182052


***Because the difference is positive, we can conclude that the layers ARE, in fact, informative.***

### Change Point Extraction

*Q6: How can I extract the bin / change point information from state_nested_layers?*

***A: It's not there. This analsys is not automatized in graph-tool. You have to create the bins yourself and check the posterior likelihood, using Eq. A3 in the 2015 paper.***

### Overlapping Blocks

*Q6: Having read Peixoto (2015), It's still not clear to me what the overlap option does or how it's called. For what purpose and how could it be integrated in this analysis?*

***A: The 'overlap' option toggles the use of the overlapping SBM, where nodes are allowed to belong to more than one group. This is also equivalent to a clustering of the 'half-edges' (i.e. edge endpoints). This would yield a different family of layered models, that could --- potentially --- uncover different kinds of layered structures. You can find more information about this here: https://dx.doi.org/10.1103/PhysRevX.5.011033***